
# 270. 컴퓨터 비전(Vision)을 위한 전이학습(Transfer Learning)
=======================================================

- 실제로 충분한 크기의 데이터셋을 갖추기는 상대적으로 드물기 때문에, (무작위 초기화를 통해) 처음부터 합성곱 신경망(Convolutional Network) 전체를 작성하는 경우는 매우 적다.   


- 대신, 매우 큰 데이터셋(예. 100가지 분류에 대해 120만개의 이미지가 포함된 ImageNet)에서 합성곱 신경망(ConvNet)을 미리 학습한 후, 이 합성곱 신경망을 관심있는 작업을 위한 초기 설정 또는 고정된 특징 추출기(fixed feature extractor)로 사용

### 전이학습의 2 가지 시나리오:

-  **합성곱 신경망의 미세조정(finetuning)**: 신경망을 ImageNet 등으로 미리 학습한 신경망으로 초기화하고 parameter 미세 조정  


-  **고정된 특징 추출기로써의 합성곱 신경망**: 마지막 완전 연결층을 제외한 모든 신경망의 가중치를 고정. 마지막의 완전 연결층은 새로운 무작위의 가중치를 갖는 계층으로 대체되어 이 계층만 학습.  


### torch 제공 pre-trained models
```
    import torchvision.models as models  
    
    resnet18 = models.resnet18()  
    alexnet = models.alexnet()  
    vgg16 = models.vgg16()  
    squeezenet = models.squeezenet1_0()  
    densenet = models.densenet161()  
    inception = models.inception_v3()  
    googlenet = models.googlenet()  
    shufflenet = models.shufflenet_v2_x1_0()  
    mobilenet = models.mobilenet_v2()  
    resnext50_32x4d = models.resnext50_32x4d()  
    wide_resnet50_2 = models.wide_resnet50_2()  
    mnasnet = models.mnasnet1_0()  
```   

데이터 불러오기
---------------

- **개미** 와 **벌** 을 분류하는 이진 분류 모델을 학습
    - 대략 120장 정도의 훈련 이미지와, 75장의 검증용 이미지를 이용
    - 전이학습을 통해 소량의 데이터로도 일반화 가능
    - ImageNet의 일부 data 이용

-  데이터를 [여기](https://download.pytorch.org/tutorial/hymenoptera_data.zip)
   에서 다운로드 받아 현재 디렉토리에 압축을 푼다

In [ ]:
# 데이터 증가(augmentation) 및 ResNet 에 적합한 normalization 적용

# ImageFolder 라이브러리

- 계층적인 폴더 구조를 가지고 있는 데이터셋을 불러올 때 사용할 수 있다. 각 이미지들이 자신의 레이블(Label) 이름으로 된 폴더 안에 들어가 있는 구조라면, ImageFolder 라이브러리를 이용하여 이를 바로 불러와 객체로 만들면 된다.

ImageFolder를 사용하기 위해선 가장 먼저 수집된 데이터의 폴더구조를 아래와 같이 설계해야한다  

최상위 경로 아래에 각각의 class name을 가지는 폴더를 구성하고 그 하위경로에 이미지가 저장되어있는 방식이다.
```
root_dir
    | --- ants/
    |      |-- 0001.jpg
    |      |-- 0002.jpg
    |      |-- ...
    | --- bees/
    |      |-- 0001.jpg
    |      |-- 0002.jpg
    |      |-- ...
    | --- rabbit/
    |      |--...
```

### `datasets.ImageFolder(folder, transform)` data loader 생성

In [ ]:
# datasets.ImageFolder를 사용하여 훈련 데이터셋을 로드
# 이 함수는 주어진 디렉토리 경로에서 이미지 파일들을 자동으로 찾아서 레이블을 할당
# data_dir은 데이터가 저장된 루트 디렉토리
# 'train' 폴더 안에 있는 이미지들은 훈련을 위해 사용
# DataLoader를 사용하여 훈련 데이터셋에 대한 배치를 준비
# - num_workers=4: 데이터를 로드하는 데 사용할 서브 프로세스 개수 (성능 최적화)
# 검증 데이터셋에 대한 DataLoader 설정

### 일부 이미지 시각화하기

- `torchvision.utils.make_grid(tensor)`  
  - `tensor (Tensor 또는 list)`: 4차원 텐서 `(B x C x H x W)` 형태의 미니 배치 텐서 또는 같은 크기의 이미지 리스트입니다. (여기서 B는 이미지 개수, C는 채널 수, H와 W는 이미지의 높이와 너비)  
  - `nrow (int, 선택)`: 한 줄에 표시할 이미지 개수입니다. 최종 그리드는 `(B / nrow, nrow)` 형태가 됩니다. 기본값은 8입니다.

In [ ]:
def imshow(inp, title=None):
    # 입력 텐서를 NumPy 배열로 변환하고 차원을 재배열합니다. PyTorch는 이미지를 [C, H, W] 형식으로 처리하지만,
    # matplotlib는 [H, W, C] 형식을 요구하므로, 차원의 순서 변경
    # 이미지를 정규화할 때 사용된 평균과 표준편차를 정의. 이 값들은 모델이 학습될 때 사용된 값들과 일치해야 합니다.
    # 정규화된 이미지 데이터를 원래의 범위로 되돌립니다.
    # np.clip 함수를 사용하여 이미지 데이터의 값이 0과 1 사이로 제한됩니다. 이는 이미지 데이터를 적절한 범위 내로 조정하기 위함입니다.
    # 제목이 있을 경우 이미지 위에 제목을 표시합니다.

In [ ]:
# 훈련 데이터 로더에서 첫 번째 배치를 가져옵니다.
# 가져온 이미지 배치를 그리드 형태로 만듭니다. nrow=2는 한 행에 이미지를 두 개씩 배치하겠다는 의미입니다.
# 만들어진 이미지 그리드를 시각화합니다.
# title에는 각 이미지의 클래스 이름을 표시하기 위해 classes 배열에서 각 클래스 인덱스에 해당하는 이름을 조회하여 리스트로 만듭니다.

Model Train
--------------

##  Fine-Tuning vs Feature Extraction 비교

|  | **Fine-Tuning (방법 1)** | **Feature Extraction (방법 2)** |
|---|-----------------|-------------------|
| **훈련되는 가중치** | 전체 네트워크 | 마지막 FC 레이어만 |
| **학습 속도** | 느림 | 빠름 |
| **필요한 데이터 크기** | 크면 좋음 | 작아도 가능 |
| **오버피팅 위험** | 있음 (데이터 적을 경우) | 적음 |
| **적응력** | 새 데이터셋에 최적화 가능 | 기존 특징을 활용 |  


    1️. 데이터셋이 크고 충분한 경우 → **Fine-Tuning**  
    🔹 학습 시간이 오래 걸리지만, 모델 성능이 더 높아질 가능성이 큼  

    2️. 데이터셋이 작고 제한적인 경우 → **Feature Extraction**  
    🔹 빠르게 학습할 수 있으며, 과적합 가능성이 낮음  


## 방법 1. pre-trained  ResNet 신경망 전체를 미세조정(finetuning)

- 미리 학습된 모델을 불러온 후 마지막의 완전 연결층 만을 새로 작성하고 **전체 parameter** 를 fine tuning

In [ ]:
# torchvision의 models 모듈에서 사전 학습된(pretrained) ResNet-18 모델을 로드
# 모델의 마지막 완전 연결(fully connected, fc) 레이어의 입력 특징(feature) 수를 조회
# 이 값은 ResNet-18 모델의 마지막 레이어에 입력되는 특징 벡터의 차원 수입니다.
# 조회한 특징 수 출력

In [ ]:
# fc layer 출력의 크기를 1000 에서 2로 변경

In [ ]:
# 모델을 현재 사용 중인 디바이스(CPU 또는 GPU)로 이동시킵니다.
# 손실 함수로 크로스 엔트로피 손실을 사용
# 모델의 모든 매개변수를 최적화 대상으로 합니다.
# SGD 사용. 학습률(lr)은 0.001, 모멘텀은 0.9로 설정합니다.
# 학습률 스케줄러 (성능이 향상되지 않을 때 자동 조정)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    # 최고의 모델 가중치를 저장하기 위한 변수. 초기값은 현재 모델의 가중치
        # 현재 에폭의 모델이 이전 에폭의 모델보다 좋은 경우, 정보를 갱신합니다.
    # 가장 나은 모델 가중치를 불러옴

In [ ]:
# 모델의 평가 모드 설정
    # 검증 데이터 로더를 통해 배치 단위로 데이터를 가져옴
        # 예측된 인덱스를 리스트에 추가
        # 실제 레이블을 리스트에 추가
    # 정답을 맞춘 케이스의 인덱스를 구함
    # 정답을 틀린 케이스의 인덱스를 구함
# 정확도를 계산하여 출력

In [ ]:
# 정답을 맞춘 데이터의 이미지를 추출하여 inputs 리스트에 추가
# 추출한 이미지들을 그리드 형태로 나열
# 이미지를 출력하기 위한 그림의 크기를 설정하고, 이미지 출력
# 정답을 틀린 데이터의 이미지를 추출하여 inputs 리스트에 추가
# 추출한 이미지들을 그리드 형태로 나열
# 이미지를 출력하기 위한 그림의 크기를 설정하고, 이미지를출력

## 방법 2 : Pre-trained ResNet을 weight 고정된 특징 추출기로 사용

- 미리 학습된 모델을 불러온 후 마지막의 **완전 연결층 만을 새로 작성**하되 마지막 계층을 제외한 **신경망의 모든 부분을 고정** (``requires_grad == False`` 로 설정)하여 ``backward()`` 중에 gradient가 계산되지 않도록 함

In [ ]:
# 사전 학습된 ResNet18 모델을 불러옴
# 모델의 모든 파라미터를 고정하여, 학습 중에 파라미터가 업데이트되지 않도록 함

In [ ]:
# 모델의 마지막 Fully Connected(FC) 레이어의 입력 특성 수를 가져옴
# # 마지막 레이어만 새롭게 학습
# 모델의 마지막 FC 레이어를 새로운 레이어로 교체하여, 출력 클래스 수를 2로 설정
# 학습률 스케줄러 (성능이 향상되지 않을 때 자동 조정)

## Train & Evaluation

In [ ]:
# 모델을 평가 모드로 설정
        # 출력값 중 최대값을 가진 인덱스를 예측값으로 선택
        # 예측값을 predictions 리스트에 추가
        # 실제 레이블을 labels 리스트에 추가
    # 예측값과 실제 레이블이 일치하는 인덱스를 찾음
    # 예측값과 실제 레이블이 불일치하는 인덱스를 찾음
# 정확도를 계산하여 출력

In [ ]:
# 정답을 맞춘 데이터의 이미지를 추출하여 inputs 리스트에 추가
# 추출한 이미지들을 그리드 형태로 나열
# 정답을 틀린 데이터의 이미지를 추출하여 inputs 리스트에 추가
# 추출한 이미지들을 그리드 형태로 나열